# PORTFOLIO OPTIMIZATION (EXTENDABLE IN SIZE) 

**COMPUTING RETURNS ON PORTFOLIO**

The optimal portfolio computed using the closing price for the last 24 months, and using the return of 3% as a constraint (after considering Microsoft's return which was the highest) is as follows - 

----------------------
MSFT: Exp ret = 0.035377, Risk = 0.062379

V: Exp ret = 0.010122, Risk = 0.084326

WMT: Exp ret = 0.010311, Risk = 0.052237

----------------------
Optimal portfolio

----------------------
x[MSFT] = 0.785487

x[V] = -0.000000

x[WMT] = 0.214513

----------------------
Exp ret = 0.030000

risk    = 0.054371

----------------------


After executing a function that takes the parameters - portfolio, last prices, budget, stock_names, and assuming our budget is 1000 USD, we spend -

1. Microsoft - 785.49 USD for 2.3 shares ~ 2 shares

2. Walmart	- 214.51 USD for 1.5 shares ~ 2 shares if possible, otherwise 1 share.

And no Visa shares are bought.

After some checking, setting a lower required return rate of 2.5%, the recommendation is this -

1. Microsoft - 586.11 USD for 1.71 shares ~ 2 shares

2. Visa - 13.7 USD for 0.06 shares ~ 0 shares

3. Walmart	- 400.19 USD for 2.8 shares ~ 3 shares if possible, otherwise 2 shares.

Microsoft still rounds up to 2 shares being purchased, and possibly 2 shares of Walmart being purchased as well. 
A 0.5 reduction in return is a lot and it may be the furthest extent to which an investor might permit the minimum return to be considered. 

If at both levels, the number of stocks purchased are basically the same, then the most sensible decision would be to invest in **2 shares of Microsoft and 2 shares of Walmart**.

**CODE WORK**

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[11]:


"""
Portfolio optimization with CVXPY
See examples at http://cvxpy.org
Author: Shabbir Ahmed
"""

import pandas as pd
import numpy as np
from cvxpy import *
import yfinance as yf

mp = dict()
stocks = ['MSFT','V','WMT']

for s in stocks:
    
    df = yf.download(s,start = '2019-12-30', end = '2021-12-30')
    df = df.iloc[df.reset_index().groupby(df.index.to_period('M'))['Date'].idxmax()]
    m = df['Adj Close'].tolist()
    mp[s] = m
    
#print(mp.keys())
#print(len(mp['V']),len(mp['WMT']),len(mp['MSFT']))


(CVXPY) Jan 15 02:31:15 AM: Encountered unexpected exception importing solver SCS:
ImportError('dlopen(/Users/Kalpana/opt/anaconda3/lib/python3.8/site-packages/_scs_direct.cpython-38-darwin.so, 2): Symbol not found: _aligned_alloc\n  Referenced from: /Users/Kalpana/opt/anaconda3/lib/python3.8/site-packages/scs/.dylibs/libgomp.1.dylib (which was built for Mac OS X 10.15)\n  Expected in: /usr/lib/libSystem.B.dylib\n in /Users/Kalpana/opt/anaconda3/lib/python3.8/site-packages/scs/.dylibs/libgomp.1.dylib')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [2]:
# read monthly_prices.csv
# compute monthly returns

def returns(mp):
    
    mr = pd.DataFrame()

    for s in mp:

        index = 0
        pr0 = mp[s][0] 
        for t in range(1,len(mp[s])):
            pr1 = mp[s][t]
            ret = (pr1-pr0)/pr0
            #mr.set_value(date,s,ret)
            mr.at[t,s]=ret
            pr0 = pr1
            
    return mr

mr = returns(mp)
def compute(mr,minret):
    
    # get symbol names
    symbols = mr.columns

    # convert monthly return data frame to a numpy matrix
    #return_data = mr.as_matrix().T
    return_data = mr.values.T

    # compute mean return
    r = np.asarray(np.mean(return_data, axis=1))

    # covariance
    C = np.asmatrix(np.cov(return_data))

    # print out expected return and std deviation
    print("----------------------")
    for j in range(len(symbols)):
        print('%s: Exp ret = %f, Risk = %f' %(symbols[j],r[j], C[j,j]**0.5))

    #set up optimization model
    n = len(symbols)
    x = Variable(n)
    req_return = minret
    ret = r.T@x
    risk = quad_form(x, C)
    prob = Problem(Minimize(risk), 
                   [sum(x) == 1, ret >= req_return, x >= 0])

    # solve problem and write solution
    try:
        prob.solve()
        print("----------------------")
        print("Optimal portfolio")
        print("----------------------")
        for s in range(len(symbols)):
            #print('x[%s] = %f'%(symbols[s],x.value[s,0]))
            print('x[%s] = %f'%(symbols[s],x.value[s]))
        print("----------------------")
        print('Exp ret = %f' %(ret.value))
        print('risk    = %f' %((risk.value)**0.5))
        print("----------------------")
    except:
        print('Error')


    return x.value


In [3]:
def actual_expense(portfolio,last_price,funds,symbols): #portfolio must be in same order as symbols
    
    expense = dict()
    shares = dict()
    leftover = funds
    
    for s in range(len(symbols)-1):
        
        fund = portfolio[s]*funds
        expense[symbols[s]]=round(fund,2)
        shares[symbols[s]]=round(fund/last_price[s],2)
        leftover -= fund
        
    expense[symbols[-1]]=round(leftover,2)
    shares[symbols[-1]]=round(leftover/last_price[-1],2)
    
    data = {'Stock':[],'Expense In USD':[],'Shares Buyable':[]}

    for k,v in shares.items():
        data['Stock'].append(k)
        data['Expense In USD'].append(expense[k])
        data['Shares Buyable'].append(v)
    
    return data

portfolio = compute(mr,0.02)
last_price = list()
symbols = mr.columns
for sym in symbols:
    
    last_price.append(mp[sym][-1])    

data = actual_expense(portfolio,last_price,1000,symbols)  
present = pd.DataFrame(data)
present


----------------------
MSFT: Exp ret = 0.035377, Risk = 0.062379
V: Exp ret = 0.010122, Risk = 0.084326
WMT: Exp ret = 0.010311, Risk = 0.052237
----------------------
Optimal portfolio
----------------------
x[MSFT] = 0.387138
x[V] = 0.080172
x[WMT] = 0.532690
----------------------
Exp ret = 0.020000
risk    = 0.046525
----------------------


,Stock,Expense In USD,Shares Buyable
0,MSFT,387.14,1.13
1,V,80.17,0.37
2,WMT,532.69,3.73
